In [65]:
import itertools
import numpy as np

# Матрицы доходов: R[action][state_from][state_to]
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}

R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

actions = [0,1,2]      # индексы действий
states  = [0,1,2]      # 0=«Отличный»,1=«Хороший»,2=«Удовлетворительный»
months = 3
state_names = ["Отличный","Хороший","Удовлетворительный"]

def expected_reward(strategy, init_state):
    # d[i] — вероятность быть в состоянии i
    d = [0.0,0.0,0.0]
    d[init_state] = 1.0
    total = 0.0
    for a in strategy:
        d_new = [0.0,0.0,0.0]
        for i in range(3):
            for j in range(3):
                pij = P[a][i][j]
                rij = R[a][i][j]
                total += d[i] * pij * rij
                d_new[j] += d[i] * pij
        d = d_new
    return total

for init in range(3):
    best = (None, -1e9)
    for strat in itertools.product(actions, repeat=months):
        er = expected_reward(strat, init_state=init)
        if er > best[1]:
            best = (strat, er)
    print(f"Начальное состояние «{state_names[init]}»:")
    print(f"  Лучшая стратегия действий: {best[0]}")
    print(f"  Ожидаемое вознаграждение: {best[1]:.2f}\n")

Начальное состояние «Отличный»:
  Лучшая стратегия действий: (1, 1, 1)
  Ожидаемое вознаграждение: 278.40

Начальное состояние «Хороший»:
  Лучшая стратегия действий: (1, 1, 1)
  Ожидаемое вознаграждение: 257.25

Начальное состояние «Удовлетворительный»:
  Лучшая стратегия действий: (2, 1, 1)
  Ожидаемое вознаграждение: 222.65



___

In [69]:
import itertools
import numpy as np

# Словари P и R по условию
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}

R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

actions = [0,1,2]      # индексы действий
states  = [0,1,2]      # 0=«Отличный»,1=«Хороший»,2=«Удовлетворительный»

def evaluate_policy(policy):
    # policy — кортеж длины 3: policy[s] = действие в состоянии s
    # 1) Собираем P_pi и r_pi
    P_pi = np.zeros((3,3))
    r_pi = np.zeros(3)
    for s in states:
        a = policy[s]
        P_pi[s,:] = P[a][s]
        # среднее вознаграждение при s и действии a
        for s2 in states:
            r_pi[s] += P[a][s][s2] * R[a][s][s2]

    # 2) Ищем стационарное распределение μ: μ = μ P_pi, sum(μ)=1
    #    Находим собственный вектор P_pi^T с собственным числом 1
    vals, vecs = np.linalg.eig(P_pi.T)
    idx = np.argmin(np.abs(vals - 1.0))
    mu = np.real(vecs[:, idx])
    mu = mu / mu.sum()

    # 3) Среднее вознаграждение по стратегии
    g = float(np.dot(mu, r_pi))
    return g, mu

best_policy = None
best_gain   = -1e9
best_mu     = None

for policy in itertools.product(actions, repeat=3):
    g, mu = evaluate_policy(policy)
    if g > best_gain:
        best_gain   = g
        best_policy = policy
        best_mu     = mu

print("Лучшая стационарная стратегия (0, 1, 2) ->", best_policy)
print("Стационарное распределение μ:", np.round(best_mu,3))
print("Среднее вознаграждение g:", round(best_gain,2))

Лучшая стационарная стратегия (0, 1, 2) -> (1, 1, 2)
Стационарное распределение μ: [0.111 0.383 0.506]
Среднее вознаграждение g: 80.86


___

In [70]:
import numpy as np

# 1) Задаём P и R по условию задачи
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}

R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

num_states    = 3
actions       = [0,1,2]    # 0=скидка,1=доставка,2=ничего
state_names   = ["Отличный","Хороший","Удовлетворительный"]
action_names  = ["3% скидка","Бесплатная доставка","Ничего"]

def evaluate_policy(policy):
    """Policy evaluation (gain g и bias h)"""
    m = num_states
    # 1) Собираем P_pi и r_pi
    P_pi = np.zeros((m,m))
    r_pi = np.zeros(m)
    for i in range(m):
        a = policy[i]
        for j in range(m):
            P_pi[i,j] = P[a][i][j]
            r_pi[i] += P[a][i][j] * R[a][i][j]

    # 2) Строим и решаем систему (m+1)x(m+1) на [h0..h_{m-1}, g]
    A = np.zeros((m+1, m+1))
    b = np.zeros(m+1)
    for i in range(m):
        A[i,i]     = 1.0
        A[i,:m]   -= P_pi[i,:]
        A[i,m]     = 1.0
        b[i]       = r_pi[i]
    # фиксация h[m-1] = 0
    A[m, m-1] = 1.0
    b[m]      = 0.0

    x = np.linalg.solve(A, b)
    h = x[:m]
    g = x[m]
    return g, h

def improve_policy(policy, h):
    """Policy improvement"""
    m = num_states
    new_pol = policy.copy()
    for i in range(m):
        best_q, best_a = -1e9, None
        for a in actions:
            # считаем Q(i,a) = r(i,a) + sum_j P[a][i][j]·h[j]
            q = 0.0
            for j in range(m):
                q += P[a][i][j] * R[a][i][j]   # r(i,a)
                q += P[a][i][j] * h[j]         # влияние bias
            if q > best_q:
                best_q, best_a = q, a
        new_pol[i] = best_a
    return new_pol

def policy_iteration():
    policy = [0]*num_states     # стартуем, например, всегда со "скидки"
    iteration = 0
    while True:
        #  -- вывод текущей политики --
        print(f"\nИтерация {iteration}. Текущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} -> {a}")

        # Оценка этой политики
        g, h = evaluate_policy(policy)
        print(f"  Оценка: средний доход g = {g:.4f}")

        # Улучшаем политику
        new_pol = improve_policy(policy, h)

        # Если не изменилось — готово
        if new_pol == policy:
            print("\nПолитика не изменилась. Алгоритм завершён.")
            break

        policy = new_pol
        iteration += 1

    return policy, g, h

if __name__ == "__main__":
    opt_policy, opt_gain, opt_h = policy_iteration()
    print("\n=== Итоговая оптимальная политика ===")
    for i, a in enumerate(opt_policy):
        print(f"{i} -> {a}")
    print(f"Оптимальный средний доход g* = {opt_gain:.4f}")


Итерация 0. Текущая политика:
  0 -> 0
  1 -> 0
  2 -> 0
  Оценка: средний доход g = 69.3778

Итерация 1. Текущая политика:
  0 -> 1
  1 -> 0
  2 -> 2
  Оценка: средний доход g = 77.9322

Итерация 2. Текущая политика:
  0 -> 1
  1 -> 1
  2 -> 2
  Оценка: средний доход g = 80.8642

Политика не изменилась. Алгоритм завершён.

=== Итоговая оптимальная политика ===
0 -> 1
1 -> 1
2 -> 2
Оптимальный средний доход g* = 80.8642


___

In [78]:
import numpy as np

# 1) Задаём P и R по условию
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}

R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

num_states   = 3
actions      = [0,1,2]     # 0=скидка,1=доставка,2=ничего
state_names  = ["Отл.","Хор.","Уд."]
action_names = ["3% скидка","доставка","ничего"]

γ = 0.9  # коэффициент дисконтирования

def evaluate_policy_discount(policy, gamma=γ):
    """
    Решаем (I - γ Pπ) V = rπ
    возвращаем вектор V размера m.
    """
    m = num_states
    Pπ = np.zeros((m,m))
    rπ = np.zeros(m)
    for i in range(m):
        a = policy[i]
        for j in range(m):
            Pπ[i,j] = P[a][i][j]
            rπ[i]  += P[a][i][j] * R[a][i][j]
    # матрица I - γ·Pπ
    A = np.eye(m) - gamma * Pπ
    V = np.linalg.solve(A, rπ)
    return V

def improve_policy_discount(policy, V, gamma=γ):
    """
    Для каждого состояния i находим a, максимизирующее
       Q(i,a) = r(i,a) + γ ∑_j P[a][i][j]·V[j]
    """
    m = num_states
    new_pol = policy.copy()
    for i in range(m):
        best_q, best_a = -1e9, None
        for a in actions:
            # r(i,a) + γ P[a][i]·V
            q = 0.0
            # мгновенное ожидание r(i,a)
            for j in range(m):
                q += P[a][i][j] * R[a][i][j]
            # плюс дисконтированное будущее
            for j in range(m):
                q += gamma * P[a][i][j] * V[j]
            if q > best_q:
                best_q, best_a = q, a
        new_pol[i] = best_a
    return new_pol

def policy_iteration_discount():
    # стартуем, например, всегда «3% скидка»
    policy = [0]*num_states
    it = 0
    while True:
        print(f"\n-- Итерация {it}, текущая политика:")
        for i, a in enumerate(policy):
            print(f"  {i} -> {a}")
        # оценка
        V = evaluate_policy_discount(policy)
        print("   V =", np.round(V,3))
        # улучшение
        new_pol = improve_policy_discount(policy, V)
        if new_pol == policy:
            print("\nПолитика не изменилась, алгоритм завершён.")
            break
        policy = new_pol
        it += 1
    return policy, V

if __name__ == "__main__":
    opt_pol, opt_V = policy_iteration_discount()
    print("\n=== Оптимальная политика ===")
    for i, a in enumerate(opt_policy):
        print(f"{i} -> {a}")
    print("Стоимость состояний V* =", np.round(opt_V,3))


-- Итерация 0, текущая политика:
  0 -> 0
  1 -> 0
  2 -> 0
   V = [734.13  713.251 655.289]

-- Итерация 1, текущая политика:
  0 -> 1
  1 -> 1
  2 -> 2
   V = [842.748 823.777 789.711]

Политика не изменилась, алгоритм завершён.

=== Оптимальная политика ===
0 -> 1
1 -> 1
2 -> 2
Стоимость состояний V* = [842.748 823.777 789.711]


___

In [80]:
import numpy as np
from scipy.optimize import linprog

# 1) Задаём матрицы P и R по условию
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}

R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

state_names  = ["Отличный","Хороший","Удовлетворительный"]
action_names = ["3% скидка","Бесплатная доставка","Ничего"]

m = 3               # число состояний
A = 3               # число действий
n_vars = m * A      # 9

# 2) Формируем вектор r (размер 9)
r = np.zeros(n_vars)
for s in range(m):
    for a in range(A):
        idx = s*A + a
        # ожидаемый доход при (s,a)
        r[idx] = sum(P[a][s][s2] * R[a][s][s2] for s2 in range(m))

# 3) Составляем матрицу A_eq и вектор b_eq
#    Первые m строк — flow-balance, последняя строка — нормировка
A_eq = np.zeros((m+1, n_vars))
b_eq = np.zeros(m+1)

# a) flow-balance: для каждого j: sum_a x_{j,a} - sum_{s,a} P[a][s][j]*x_{s,a} = 0
for j in range(m):
    for s in range(m):
        for a in range(A):
            idx = s*A + a
            if s == j:
                A_eq[j, idx] += 1.0
            A_eq[j, idx] -= P[a][s][j]

# b) нормировка sum x_{s,a} = 1
A_eq[m, :] = 1.0
b_eq[m]    = 1.0

# 4) решаем LP: maximize r^T x  <=>  minimize -r^T x
res = linprog(
    c = -r,
    A_eq = A_eq,
    b_eq = b_eq,
    bounds = [(0, None)] * n_vars,
    method = 'highs'      # можно попробовать 'revised simplex'
)

if not res.success:
    raise RuntimeError("LP не сошлось: " + res.message)

x_opt = res.x
g_opt = r.dot(x_opt)

# 5) Восстанавливаем политику: для каждого состояния берем аргмакс по x_{s,a}
policy = []
for s in range(m):
    values = [ x_opt[s*A + a] for a in range(A) ]
    best_a = int(np.argmax(values))
    policy.append(best_a)

# 6) Выводим результат
print(f"Оптимальное среднее вознаграждение g* = {g_opt:.4f}\n")
print("Оптимальная стан. детермин. политика:")
for s in range(m):
    print(f"  {s} → {policy[s]}")

Оптимальное среднее вознаграждение g* = 80.8642

Оптимальная стан. детермин. политика:
  0 → 1
  1 → 1
  2 → 2


___

In [81]:
import numpy as np
from scipy.optimize import linprog

# 1) Данные задачи
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}
R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

m = 3       # число состояний
A = 3       # число действий
gamma = 0.9 # дисконт-фактор
# стартуем из состояния "Отличный"
d0 = np.array([1.0, 0.0, 0.0])

# 2) Построим вектор r размером m*A
r = np.zeros(m*A)
for s in range(m):
    for a in range(A):
        idx = s*A + a
        # ожидаемый доход при паре (s,a)
        r[idx] = sum(P[a][s][j] * R[a][s][j] for j in range(m))

# 3) Матрица равенств A_eq y = b_eq
#    m уравнений — по каждому состоянию j
A_eq = np.zeros((m, m*A))
b_eq = d0.copy()

for j in range(m):
    for s in range(m):
        for a in range(A):
            idx = s*A + a
            # левая часть: + y[j,a]
            if s == j:
                A_eq[j, idx] += 1.0
            # и −γ·P[a][s][j]·y[s,a]
            A_eq[j, idx] -= gamma * P[a][s][j]

# 4) Решаем LP: maximize r^T y  <=>  minimize -r^T y
res = linprog(
    c      = -r,
    A_eq   = A_eq,
    b_eq   = b_eq,
    bounds = [(0, None)] * (m*A),
    method = 'highs'
)
if not res.success:
    raise RuntimeError("LP не сошлось: " + res.message)

y_opt = res.x
V_opt = r.dot(y_opt)  # оптимальный дисконтированный доход

# 5) Восстанавливаем стратегию: для каждого s берём argmax_a y_opt[s,a]
policy = []
for s in range(m):
    arr = [y_opt[s*A + a] for a in range(A)]
    policy.append(int(np.argmax(arr)))

# 6) Выводим
state_names  = ["Отличный","Хороший","Удовлетворительный"]
action_names = ["3% скидка","Бесплатная доставка","Ничего"]
print(f"Оптимальный дисконтированный доход V* = {V_opt:.4f}\n")
print("Оптимальная политика:")
for s,a in enumerate(policy):
    print(f"{s} → {a}")

Оптимальный дисконтированный доход V* = 842.7485

Оптимальная политика:
0 → 1
1 → 1
2 → 2


___


In [82]:
import numpy as np
import pulp

# 1) Данные MDP
P = {
    0: [[0.3,0.5,0.2],
        [0.2,0.6,0.2],
        [0.1,0.2,0.7]],
    1: [[0.2,0.7,0.1],
        [0.1,0.4,0.5],
        [0.1,0.2,0.7]],
    2: [[0.3,0.4,0.3],
        [0.2,0.6,0.2],
        [0.1,0.3,0.6]],
}
R = {
    0: [[110,100,70],
        [100, 80,50],
        [ 80, 60,40]],
    1: [[120,100,70],
        [110,100,90],
        [100, 70,60]],
    2: [[110, 80,50],
        [100, 60,40],
        [ 80, 70,60]],
}

m     = 3               # состояний
A     = 3               # действий
gamma = 0.9             # дисконт
d0    = [1.0, 0.0, 0.0]  # стартуем из «Отличного»

# 2) Предсчитаем «моментный» r[s,a]
r = {}
for s in range(m):
    for a in range(A):
        r[s,a] = sum(P[a][s][j] * R[a][s][j] for j in range(m))

# 3) Формируем LP через PuLP
model = pulp.LpProblem("Discounted_MDP", pulp.LpMaximize)

# переменные y[s,a] >= 0
y = {(s,a): pulp.LpVariable(f"y_{s}_{a}", lowBound=0) 
     for s in range(m) for a in range(A)}

# --- целевая функция: max sum r[s,a]*y[s,a]
model += pulp.lpSum(r[s,a] * y[s,a] for s in range(m) for a in range(A)), "Obj"

# --- баланс: для каждого j: sum_a y[j,a] - γ sum_{s,a} P[a][s][j]*y[s,a] == d0[j]
for j in range(m):
    expr = pulp.lpSum(y[j,a] for a in range(A)) \
           - gamma * pulp.lpSum(P[a][s][j] * y[s,a] 
                               for s in range(m) for a in range(A))
    model += (expr == d0[j], f"flow_state_{j}")

# --- решаем
model.solve(pulp.PULP_CBC_CMD(msg=False))

# 4) Собираем результаты
print("=== Результат LP ===")
print("Status:", pulp.LpStatus[model.status])
print("Оптимальный V* = ", pulp.value(model.objective))
print()

# оптимальная политика
print("Оптимальная политика:")
for s in range(m):
    best_a, best_val = None, -1e9
    for a in range(A):
        val = y[s,a].value()
        if val > best_val:
            best_val, best_a = val, a
    print(f"  Состояние {s} → действие {best_a}  (y={best_val:.4f})")
print()

# 5) Анализ чувствительности
print("=== Dual variables (shadow prices) для уравнений потока ===")
for cname, con in model.constraints.items():
    # .pi — дуальная переменная (shadow price)
    print(f"  {cname:15s}  π = {con.pi: .4f}")

print()
print("=== Reduced costs для y[s,a] ===")
for (s,a), var in y.items():
    # var.dj — reduced cost
    print(f"  y[{s},{a}] = {var.value():.4f}    reduced cost = {var.dj: .4f}")

=== Результат LP ===
Status: Optimal
Оптимальный V* =  842.7484618000001

Оптимальная политика:
  Состояние 0 → действие 1  (y=2.0879)
  Состояние 1 → действие 1  (y=3.7930)
  Состояние 2 → действие 2  (y=4.1191)

=== Dual variables (shadow prices) для уравнений потока ===
  flow_state_0     π =  842.7485
  flow_state_1     π =  823.7773
  flow_state_2     π =  789.7114

=== Reduced costs для y[s,a] ===
  y[0,0] = 0.0000    reduced cost = -5.3585
  y[0,1] = 2.0879    reduced cost =  0.0000
  y[0,2] = 0.0000    reduced cost = -25.4245
  y[1,0] = 0.0000    reduced cost = -7.0948
  y[1,1] = 3.7930    reduced cost = -0.0000
  y[1,2] = 0.0000    reduced cost = -21.0948
  y[2,0] = 0.0000    reduced cost = -20.0659
  y[2,1] = 0.0000    reduced cost = -2.0659
  y[2,2] = 4.1191    reduced cost = -0.0000
